# 基于 Keras Applications 的预训练模型在隐语联邦学习环境下的微调
## 引言
预训练模型加载和精调在机器学习中非常重要。一般来说，从头训练一个非常大的模型，不仅需要大量的算力资源，同时也需要耗费大量的时间。所以在传统的机器学习中，使用预训练模型，然后针对具体的任务做微调和迁移学习非常普遍。同样的，对于联邦学习来说，如果能够加载预训练模型进行微调和迁移学习，不仅能够节省参与方的算力资源，降低参与方的准入门槛，同时也能够加快模型的学习速度。

得益于隐语联邦学习模块优异的兼容性，使得其可以直接加载TensorFlow.Keras的一系列[预训练模型](https://keras.io/api/applications/)；本教程将基于TensorFlow.Keras的[InceptionV3](https://arxiv.org/abs/1512.00567)的[微调教程](https://keras.io/api/applications/#finetune-inceptionv3-on-a-new-set-of-classes)展现如何基于TensorFlow.Keras的预训练模型在SecretFlow的框架下进行微调，充分展现SecretFlow的易用性。

## 加载数据集
### 数据集介绍
Flower 数据集介绍：flower 数据集是一个包含了 5 种花卉（雏菊、蒲公英、玫瑰、向日葵、郁金香）共计 4323 张彩色图片的数据集。每种花卉都有多个角度和不同光照下的图片，每张图片的分辨率为 320x240。这个数据集常用于图像分类和机器学习算法的训练与测试。数据集中每个类别的数量分别是：daisy（633），dandelion（898），rose（641），sunflower（699），tulip（852）

下载地址: http://download.tensorflow.org/example_images/flower_photos.tgz

### 下载数据集并解压

In [1]:
import tempfile
import tensorflow as tf


_temp_dir = tempfile.mkdtemp()
path_to_flower_dataset = tf.keras.utils.get_file(
    "flower_photos",
    "https://secretflow-data.oss-accelerate.aliyuncs.com/datasets/tf_flowers/flower_photos.tgz",
    untar=True,
    cache_dir=_temp_dir,
)

2023-09-24 13:59:20.552488: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 13:59:20.727881: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-24 13:59:20.732410: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-24 13:59:20.732427: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

67588319/67588319 [==============================] - 1s 0us/step


### 加载数据集

In [2]:
import math
import tensorflow as tf

img_height = 180
img_width = 180
batch_size = 32
# In this example, we use the TensorFlow interface for development.
data_set = tf.keras.utils.image_dataset_from_directory(
    path_to_flower_dataset,
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

Found 1201 files belonging to 5 classes.
Using 961 files for training.
Using 240 files for validation.


2023-09-24 13:59:30.293178: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-24 13:59:30.293269: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-09-24 13:59:30.293312: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-09-24 13:59:30.293351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-09-24 13:59:30.436001: W tensorfl

### 划分数据集

In [3]:
train_set = data_set[0]
test_set = data_set[1]

### 查看数据集


In [4]:
print(type(train_set), type(test_set))

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'> <class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


In [5]:
x, y = next(iter(train_set))
print(f"x.shape = {x.shape}")
print(f"y.shape = {y.shape}")

x.shape = (32, 180, 180, 3)
y.shape = (32,)


## 单机模式进行微调
单机模式下进行预训练模型的微调，基本上参考TensorFlow.Keras的[官方教程](https://keras.io/api/applications/#finetune-inceptionv3-on-a-new-set-of-classes)，并根据数据集格式在编译模型的参数上作适当的修改，但影响不大；

### 微调顶部分类器

In [6]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 10 classes
predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"],
              )

In [7]:
# train the model on the new data for a few epochs
model.fit(train_set, validation_data=test_set, epochs=10)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

Epoch 1/10
31/31 [==============================] - 19s 486ms/step - loss: 103.8929 - accuracy: 0.2393 - val_loss: 36.4069 - val_accuracy: 0.2458
Epoch 2/10
31/31 [==============================] - 13s 426ms/step - loss: 27.2670 - accuracy: 0.2674 - val_loss: 30.7927 - val_accuracy: 0.1792
Epoch 3/10
31/31 [==============================] - 13s 431ms/step - loss: 19.9083 - accuracy: 0.2945 - val_loss: 37.4682 - val_accuracy: 0.2583
Epoch 4/10
31/31 [==============================] - 13s 421ms/step - loss: 15.3037 - accuracy: 0.2955 - val_loss: 12.0967 - val_accuracy: 0.2583
Epoch 5/10
31/31 [==============================] - 13s 425ms/step - loss: 11.6578 - accuracy: 0.3205 - val_loss: 6.7864 - val_accuracy: 0.3667
Epoch 6/10
31/31 [==============================] - 13s 422ms/step - loss: 8.8003 - accuracy: 0.3351 - val_loss: 11.5285 - val_accuracy: 0.3000
Epoch 7/10
31/31 [==============================] - 13s 422ms/step - loss: 7.5333 - accuracy: 0.3444 - val_loss: 11.9257 - val_accu

### 冻结底层微调顶层网络

In [8]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_14
5

In [9]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [10]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='sparse_categorical_crossentropy', metrics=["accuracy"],)

In [11]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(train_set, validation_data=test_set, epochs=10)

Epoch 1/10
31/31 [==============================] - 20s 508ms/step - loss: 1.9437 - accuracy: 0.2810 - val_loss: 1.7874 - val_accuracy: 0.2583
Epoch 2/10
31/31 [==============================] - 14s 464ms/step - loss: 1.8572 - accuracy: 0.3143 - val_loss: 1.8464 - val_accuracy: 0.3250
Epoch 3/10
31/31 [==============================] - 15s 467ms/step - loss: 1.7950 - accuracy: 0.3226 - val_loss: 1.8715 - val_accuracy: 0.3167
Epoch 4/10
31/31 [==============================] - 14s 464ms/step - loss: 1.7486 - accuracy: 0.3215 - val_loss: 1.8644 - val_accuracy: 0.3333
Epoch 5/10
31/31 [==============================] - 15s 467ms/step - loss: 1.7059 - accuracy: 0.3278 - val_loss: 1.7934 - val_accuracy: 0.3250
Epoch 6/10
31/31 [==============================] - 15s 468ms/step - loss: 1.6775 - accuracy: 0.3403 - val_loss: 1.7646 - val_accuracy: 0.3208
Epoch 7/10
31/31 [==============================] - 14s 466ms/step - loss: 1.6493 - accuracy: 0.3267 - val_loss: 1.7183 - val_accuracy: 0.3000

### 单机模式小结
以上我们按照官方教程在数据集Flower 上成功微调了 InceptionV3 模型。接下来我们将展示如何将单机模式下的微调拓展到联邦学习模式下进行微调。

## 联邦学习模式下进行微调

### 环境设置
首先我们初始化各个参与方。

In [12]:
%load_ext autoreload
%autoreload 2

In [13]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()
sf.init(['alice', 'bob', 'charlie'], address="local", log_to_driver=False)
alice, bob, charlie = sf.PYU('alice'), sf.PYU('bob'), sf.PYU('charlie')

The version of SecretFlow: 1.2.0.dev20230918


2023-09-24 14:04:25,229	INFO worker.py:1538 -- Started a local Ray instance.


### 定义Dataloader
我们可以参考[TensorFlow下的DataBuilder教程](https://www.secretflow.org.cn/docs/secretflow/latest/zh-Hans/tutorial/CustomDataLoaderTF)定义我们自己的DataBuilder。

In [14]:
def create_dataset_builder(
    batch_size=32,
):
    def dataset_builder(folder_path, stage="train"):
        import math

        import tensorflow as tf

        img_height = 180
        img_width = 180
        data_set = tf.keras.utils.image_dataset_from_directory(
            folder_path,
            validation_split=0.2,
            subset="both",
            seed=123,
            image_size=(img_height, img_width),
            batch_size=batch_size,
        )
        if stage == "train":
            train_dataset = data_set[0]
            train_step_per_epoch = math.ceil(len(data_set[0].file_paths) / batch_size)
            return train_dataset, train_step_per_epoch
        elif stage == "eval":
            eval_dataset = data_set[1]
            eval_step_per_epoch = math.ceil(len(data_set[1].file_paths) / batch_size)
            return eval_dataset, eval_step_per_epoch

    return dataset_builder

In [15]:
data_builder_dict = {
    alice: create_dataset_builder(
        batch_size=32,
    ),
    bob: create_dataset_builder(
        batch_size=32,
    ),
}

### 定义 SecureAggregator

In [16]:
from secretflow.ml.nn import FLModel
from secretflow.security.aggregation import SecureAggregator

device_list = [alice, bob]
aggregator = SecureAggregator(charlie, [alice, bob])

INFO:root:Create proxy actor <class 'secretflow.security.aggregation.secure_aggregator._Masker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.security.aggregation.secure_aggregator._Masker'> with party bob.


### 定义数据加载路径
为了简便起见，我们在 单机模拟模式下直接加载同一处路径所对应的数据集

In [17]:
data = {
    alice: path_to_flower_dataset,
    bob: path_to_flower_dataset,
}

### 微调顶部分类器
我们只要参照教程里对模型的定义，在函数里完成我们对模型的定义即可；可以看到代码几乎不需要作任何修改，只需要进行适当的封装。
为了方便作对比实验，我们额外添加是否加载权重的选项。

In [18]:
def create_inception_v3_model_classifier(num_classes, is_load_weight=True):
    def create_model():
        from tensorflow import keras

        # Create model
        # create the base pre-trained model
        if is_load_weight:
            base_model = InceptionV3(weights='imagenet', include_top=False)
        else:
            base_model = InceptionV3(weights=None, include_top=False)

        # add a global spatial average pooling layer
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        # let's add a fully-connected layer
        x = Dense(1024, activation='relu')(x)
        # and a logistic layer -- let's say we have 10 classes
        predictions = Dense(num_classes, activation='softmax')(x)

        # this is the model we will train
        model = Model(inputs=base_model.input, outputs=predictions)

        # first: train only the top layers (which were randomly initialized)
        # i.e. freeze all convolutional InceptionV3 layers
        for layer in base_model.layers:
            layer.trainable = False

        
        # Compile model
        model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"],
              )
        
        return model

    return create_model

#### 加载预训练模型权重并且微调

In [19]:
# prepare model
num_classes = 5

# keras model
weight_model = create_inception_v3_model_classifier(num_classes=num_classes,  is_load_weight=True)


fed_model = FLModel(
    device_list=device_list,
    model=weight_model,
    aggregator=aggregator,
    backend="tensorflow",
    strategy="fed_avg_w",
    random_seed=1234,
)

INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party bob.


In [20]:
history = fed_model.fit(
    data,
    None,
    validation_data=data,
    epochs=5,
    batch_size=32,
    aggregate_freq=2,
    sampler_method="batch",
    random_seed=1234,
    dp_spent_step_freq=1,
    dataset_builder=data_builder_dict,
)

INFO:root:FL Train Params: {'self': <secretflow.ml.nn.fl.fl_model.FLModel object at 0x7fb758663c70>, 'x': {PYURuntime(alice): '/tmp/tmpy35ta2f9/datasets/flower_photos', PYURuntime(bob): '/tmp/tmpy35ta2f9/datasets/flower_photos'}, 'y': None, 'batch_size': 32, 'batch_sampling_rate': None, 'epochs': 5, 'verbose': 1, 'callbacks': None, 'validation_data': {PYURuntime(alice): '/tmp/tmpy35ta2f9/datasets/flower_photos', PYURuntime(bob): '/tmp/tmpy35ta2f9/datasets/flower_photos'}, 'shuffle': False, 'class_weight': None, 'sample_weight': None, 'validation_freq': 1, 'aggregate_freq': 2, 'label_decoder': None, 'max_batch_size': 20000, 'prefetch_buffer_size': None, 'sampler_method': 'batch', 'random_seed': 1234, 'dp_spent_step_freq': 1, 'audit_log_dir': None, 'dataset_builder': {PYURuntime(alice): <function create_dataset_builder.<locals>.dataset_builder at 0x7fb6f0076ee0>, PYURuntime(bob): <function create_dataset_builder.<locals>.dataset_builder at 0x7fb4957a1670>}, 'wait_steps': 100}
32it [01:40

#### 只加载网络结构同时随机初始化

In [21]:
# keras model
no_weight_model = create_inception_v3_model_classifier(num_classes=num_classes,  is_load_weight=False)


fed_model = FLModel(
    device_list=device_list,
    model=no_weight_model ,
    aggregator=aggregator,
    backend="tensorflow",
    strategy="fed_avg_w",
    random_seed=1234,
)

INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party bob.


In [22]:
history = fed_model.fit(
    data,
    None,
    validation_data=data,
    epochs=5,
    batch_size=32,
    aggregate_freq=2,
    sampler_method="batch",
    random_seed=1234,
    dp_spent_step_freq=1,
    dataset_builder=data_builder_dict,
)

INFO:root:FL Train Params: {'self': <secretflow.ml.nn.fl.fl_model.FLModel object at 0x7fb7586491c0>, 'x': {PYURuntime(alice): '/tmp/tmpy35ta2f9/datasets/flower_photos', PYURuntime(bob): '/tmp/tmpy35ta2f9/datasets/flower_photos'}, 'y': None, 'batch_size': 32, 'batch_sampling_rate': None, 'epochs': 5, 'verbose': 1, 'callbacks': None, 'validation_data': {PYURuntime(alice): '/tmp/tmpy35ta2f9/datasets/flower_photos', PYURuntime(bob): '/tmp/tmpy35ta2f9/datasets/flower_photos'}, 'shuffle': False, 'class_weight': None, 'sample_weight': None, 'validation_freq': 1, 'aggregate_freq': 2, 'label_decoder': None, 'max_batch_size': 20000, 'prefetch_buffer_size': None, 'sampler_method': 'batch', 'random_seed': 1234, 'dp_spent_step_freq': 1, 'audit_log_dir': None, 'dataset_builder': {PYURuntime(alice): <function create_dataset_builder.<locals>.dataset_builder at 0x7fb6f0076ee0>, PYURuntime(bob): <function create_dataset_builder.<locals>.dataset_builder at 0x7fb4957a1670>}, 'wait_steps': 100}
32it [01:38

### 冻结底层微调顶层网络
我们只要参照教程里对模型的定义，在函数里完成我们对模型的定义即可；可以看到代码几乎不需要作任何修改，只需要进行适当的封装。
为了方便作对比实验，我们额外添加是否加载权重的选项。

In [23]:
def create_inception_v3_model_fine_tune(num_classes, is_load_weight = True):
    def create_model():
        from tensorflow import keras

        # Create model
        # create the base pre-trained model
        if is_load_weight:
            base_model = InceptionV3(weights='imagenet', include_top=False)
        else:
            base_model = InceptionV3(weights=None, include_top=False)


        # add a global spatial average pooling layer
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        # let's add a fully-connected layer
        x = Dense(1024, activation='relu')(x)
        # and a logistic layer -- let's say we have 10 classes
        predictions = Dense(num_classes, activation='softmax')(x)

        # this is the model we will train
        model = Model(inputs=base_model.input, outputs=predictions)

        for layer in model.layers[:249]:
            layer.trainable = False
        for layer in model.layers[249:]:
            layer.trainable = True

        
        # Compile model
        model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='sparse_categorical_crossentropy', metrics=["accuracy"],)
        
        return model

    return create_model

#### 加载预训练模型权重并且微调

In [24]:
# keras model
weight_model = create_inception_v3_model_fine_tune(num_classes=num_classes, is_load_weight=True)


fed_model = FLModel(
    device_list=device_list,
    model=weight_model,
    aggregator=aggregator,
    backend="tensorflow",
    strategy="fed_avg_w",
    random_seed=1234,
)

INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party bob.


In [26]:
history = fed_model.fit(
    data,
    None,
    validation_data=data,
    epochs=5,
    batch_size=32,
    aggregate_freq=2,
    sampler_method="batch",
    random_seed=1234,
    dp_spent_step_freq=1,
    dataset_builder=data_builder_dict,
)

INFO:root:FL Train Params: {'self': <secretflow.ml.nn.fl.fl_model.FLModel object at 0x7fb75869e340>, 'x': {PYURuntime(alice): '/tmp/tmpy35ta2f9/datasets/flower_photos', PYURuntime(bob): '/tmp/tmpy35ta2f9/datasets/flower_photos'}, 'y': None, 'batch_size': 32, 'batch_sampling_rate': None, 'epochs': 5, 'verbose': 1, 'callbacks': None, 'validation_data': {PYURuntime(alice): '/tmp/tmpy35ta2f9/datasets/flower_photos', PYURuntime(bob): '/tmp/tmpy35ta2f9/datasets/flower_photos'}, 'shuffle': False, 'class_weight': None, 'sample_weight': None, 'validation_freq': 1, 'aggregate_freq': 2, 'label_decoder': None, 'max_batch_size': 20000, 'prefetch_buffer_size': None, 'sampler_method': 'batch', 'random_seed': 1234, 'dp_spent_step_freq': 1, 'audit_log_dir': None, 'dataset_builder': {PYURuntime(alice): <function create_dataset_builder.<locals>.dataset_builder at 0x7fb6f0076ee0>, PYURuntime(bob): <function create_dataset_builder.<locals>.dataset_builder at 0x7fb4957a1670>}, 'wait_steps': 100}


32it [01:40,  3.15s/it, epoch: 1/5 -  loss:1.3916817903518677  accuracy:0.433925062417984  val_loss:1.4990912675857544  val_accuracy:0.3499999940395355 ]
32it [01:40,  3.13s/it, epoch: 2/5 -  loss:1.3310511112213135  accuracy:0.48376351594924927  val_loss:1.47920823097229  val_accuracy:0.36666667461395264 ]
32it [01:39,  3.10s/it, epoch: 3/5 -  loss:1.3042552471160889  accuracy:0.49791839718818665  val_loss:1.4480222463607788  val_accuracy:0.36250001192092896 ]
32it [01:38,  3.09s/it, epoch: 4/5 -  loss:1.2815332412719727  accuracy:0.5054121613502502  val_loss:1.4403711557388306  val_accuracy:0.34583333134651184 ]
32it [01:39,  3.12s/it, epoch: 5/5 -  loss:1.249996542930603  accuracy:0.5104079842567444  val_loss:1.420535683631897  val_accuracy:0.3583333194255829 ]


#### 只加载网络结构同时随机初始化

In [27]:
# keras model
no_weight_model = create_inception_v3_model_fine_tune(num_classes=num_classes, is_load_weight=False)


fed_model = FLModel(
    device_list=device_list,
    model=no_weight_model,
    aggregator=aggregator,
    backend="tensorflow",
    strategy="fed_avg_w",
    random_seed=1234,
)

INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party bob.


In [28]:
history = fed_model.fit(
    data,
    None,
    validation_data=data,
    epochs=5,
    batch_size=32,
    aggregate_freq=2,
    sampler_method="batch",
    random_seed=1234,
    dp_spent_step_freq=1,
    dataset_builder=data_builder_dict,
)

INFO:root:FL Train Params: {'self': <secretflow.ml.nn.fl.fl_model.FLModel object at 0x7fb6d8261160>, 'x': {PYURuntime(alice): '/tmp/tmpy35ta2f9/datasets/flower_photos', PYURuntime(bob): '/tmp/tmpy35ta2f9/datasets/flower_photos'}, 'y': None, 'batch_size': 32, 'batch_sampling_rate': None, 'epochs': 5, 'verbose': 1, 'callbacks': None, 'validation_data': {PYURuntime(alice): '/tmp/tmpy35ta2f9/datasets/flower_photos', PYURuntime(bob): '/tmp/tmpy35ta2f9/datasets/flower_photos'}, 'shuffle': False, 'class_weight': None, 'sample_weight': None, 'validation_freq': 1, 'aggregate_freq': 2, 'label_decoder': None, 'max_batch_size': 20000, 'prefetch_buffer_size': None, 'sampler_method': 'batch', 'random_seed': 1234, 'dp_spent_step_freq': 1, 'audit_log_dir': None, 'dataset_builder': {PYURuntime(alice): <function create_dataset_builder.<locals>.dataset_builder at 0x7fb6f0076ee0>, PYURuntime(bob): <function create_dataset_builder.<locals>.dataset_builder at 0x7fb4957a1670>}, 'wait_steps': 100}
32it [01:42

### 联邦学习小结
可以看到，对照着TensorFlow的官方教程，隐语能够无缝地兼容所给出的微调方式；并且我们可以看到，通过对预训练模型的兼容，我们可以不需要自己再重新写出复杂网络的模型结构，InceptionV3 的网络结构源代码位于：[source code of Inception V3](https://github.com/keras-team/keras/blob/v2.13.1/keras/applications/inception_v3.py)，并且通过对比实验我们可以看出，加载预训练模型的权重，可以让我们的模型性能更优秀。

## 总结
本篇教程，我们以Inception V3为例介绍了如何在隐语的联邦学习模式下基于直接加载TensorFlow.Keras的[预训练模型]，通过直接加载预训练模型，我们能够获得：
- 不需要再次编写复杂模型的结构代码
- 基于预训练模型进行微调和迁移学习
- 使用预训练权重模型能够使得联邦模型获得更好的性能